import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
import math  
import time

#def U(n, x):
#    if n == 0: return 1
#    if n == 1: return 2 * x
#    return 2 * x * U(n - 1, x) - U(n - 2, x)

def U(n, x):
    if math.sqrt(1 - x*x) == 0:
        return n + 1
    else:
        return math.sin((n + 1) * math.acos(x)) / math.sqrt(1 - x*x)

def f(εi, c, N):
    f = 0
    for j in range(0, N):
        f += c[j] * U(j, εi)
    return f

def SME(A, x, σ, M, N):
    f = np.matmul(A, x)
    totalSum = 0
    for i in range(0, M):
        totalSum += (f[i] - σ[i]) * (f[i] - σ[i])
    return math.sqrt(totalSum / M)

# def SVD(A, b, N):
#     X, S, V = np.linalg.svd(A)
#     return V.T @ np.linalg.inv(np.diag(S)) @ X[:, 0:N].T @ b

def CreateA(ε, n):
    A = np.zeros((len(ε), n))
    for i in range(0, len(ε)):
        for j in range(0, n):
            A[i][j] = U(j, ε[i])
    return A

def Givens(AA, bb, N):
    b = np.copy(bb)
    A = np.copy(AA)
    # Цикл по стобцам.
    for i in range(0, N):
        # Ищем индекс максимального элемента.
        max = 0
        j = -1
        for s in range(i + 1, len(A)):
            if math.fabs(A[s][i]) > max:
                max = math.fabs(A[s][i])
                j = s
        ai = np.copy(A[i][:])
        aj = np.copy(A[j][:])
        A[i][:] = aj
        A[j][:] = ai
        b1 = np.copy(b[i])
        b2 = np.copy(b[j])
        b[i] = b2
        b[j] = b1
         # Количество итераций в столбце.
        for p in range(i + 1, len(A)):
            j = p
            # Берем i-ю и j-ю строки.
            ai = np.copy(A[i][:])
            aj = np.copy(A[p][:])
            denominator = math.sqrt(ai[i]*ai[i] + aj[i]*aj[i])
            if denominator != 0:
                cos =  ai[i] / denominator
                sin = -aj[i] / denominator
                A[i] = ai*cos - aj*sin
                A[j] = ai*sin + aj*cos
                b1 = b[i]*cos - b[j]*sin
                b2 = b[i]*sin + b[j]*cos
                b[i] = b1
                b[j] = b2
    N = N - 1
    X = np.zeros(N + 1)
    for i in range(0, N + 1):
        sum = b[N - i]
        for j in range(0, i):
            sum -= A[N - i][N - j] * X[N - j]
        X[N - i] = sum / A[N - i][N - i]
    return X, SME(AA, X, bb, len(A), N)

def FindDiag(B):
    # Find B = V D^2 V^t.
    max = 1.0
    n = len(B)
    V = np.zeros((n, n))
    # Изначально можно считать, что B = I B I, где I - единичная, 
    # потом итерационно B приведем к диагональной матрице D, а V к ортогональной матрице перехода.
    for i in range(0, n):
        V[i][i] = 1.0
    # В качестве нормы я беру максимум, а не среднеквадратичное, близкое к нулю - меньше 10^{-8}.
    while max > 10**(-12):
        # Ищем максимальный элемент, который занулим + это норма.
        max = 0
        θ = 0
        i = -1
        j = -1
        for p in range(0, n):
            for q in range(0, n):
                if p != q:
                    if math.fabs(B[p][q]) > max:
                        max = math.fabs(B[p][q])
                        i = p
                        j = q
        # Считаем угол, для зануления.
        if B[j][j] == B[i][i]:
          θ = math.pi / 4.0
        else:
          r = 2 * B[i][j] / (B[j][j] - B[i][i])
          θ = math.atan(r) / 2.0
        s = math.sin(θ)
        c = math.cos(θ)
        # Копируем j и k - ю строки (матрица симметрична).
        bi = np.copy(B[i][:])
        bj = np.copy(B[j][:])
        # Теперь пересчитаем все элементы, это 2 строки и 2 столбца, отдельно их пересечения.
        B[i][:] = c * bi - s * bj
        B[j][:] = s * bi + c * bj
        B[:][i] = np.copy(B[i][:])
        B[:][j] = np.copy(B[j][:])
        B[i][i] = c * c * bi[i] - 2 * s * c * bi[j] + s * s * bj[j]
        B[j][j] = s * s * bi[i] + 2 * s * c * bi[j] + c * c * bj[j]
        X = (c * c - s * s) * bi[j] + s * c * (bi[i] - bj[j])
        B[i][j] = X
        B[j][i] = X
        # Мы сделали итерацию по изменению матрицы, В сойдется к диагональной, 
        # нужно пересчитать матрицу перехода V как  V = V * матрица вращения.
        vi = np.copy(V[:][i])
        vj = np.copy(V[:][j])
        V[:][i] = c * vi - s * vj
        V[:][j] = s * vi + c * vj
    return np.diag(np.diag(B)), V

# def FindDiag(B):
#     max = 1.0
#     n = len(B)
#     V = np.zeros((n, n))
#     for i in range(0, n):
#         V[i][i] = 1.0
#     while max > 0.1**8:
#         max = 0
#         θ = 0
#         j = -1
#         k = -1
#         for p in range(0, n):
#           for q in range(0, n):
#             if p != q:
#               if math.fabs(B[p][q]) > max:
#                 max = math.fabs(B[p][q])
#                 j = p
#                 k = q
#         if B[j][j] == B[k][k]:
#             θ = math.pi / 4.0
#         else:
#             θ = math.atan(2 * B[j][k] / (B[j][j] - B[k][k])) / 2.0
#         s = math.sin(θ)
#         c = math.cos(θ)
#         aj = np.copy(B[j][:])
#         ak = np.copy(B[k][:])
#         B[j][:] = c * aj - s * ak
#         B[k][:] = s * aj + c * ak
#         B[:][j] = np.copy(B[j][:])
#         B[:][k] = np.copy(B[k][:])
#         B[j][j] = c * c * aj[j] - 2 * s * c * aj[k] + s * s * ak[k]
#         B[k][k] = s * s * aj[j] + 2 * s * c * aj[k] + c * c * ak[k]
#         value = (c * c - s * s) * aj[k] + s * c * (ak[k] - aj[j])
#         B[j][k] = value
#         B[k][j] = value
#         vj = np.copy(V[:][j])
#         vk = np.copy(V[:][k])
#         V[:][j] = c * vj - s * vk
#         V[:][k] = c * vj + s * vk
#     return np.diag(np.diag(B)), V

def SVD(A, b, N):
    Sigma, V = FindDiag(A.T @ A) # NxN matrix, B := A^tA.
    Sigma = np.sqrt(Sigma)
    Sigma_1 = np.linalg.inv(Sigma)
    U = A @ V @ Sigma_1
    return V @ Sigma_1 @ U.T @ b

# Программа (main)
# Иницилизация
# Я ввожу N и работаю с N + 1, т.е. при N = 0 получаю полином степени 1 и т.д.
FileNumber = 4
N = 2
Input = np.loadtxt("/content/4.txt")
ε = Input[:, 0]
b = Input[:, 1]

# find max
σ_max = 0
for i in range(0, len(ε)):
    if math.fabs(b[i]) > σ_max:
        σ_max = math.fabs(b[i])

for g in range(0, 5):
    N = g
    A = CreateA(ε, N + 1)

    # QR
    # start_time = time.time()
    # Xqr, SMEqr = Givens(A, b, N + 1)

    # print("Обусловленность А %e" % np.linalg.cond(A) + "|||| SMEqr = %e" % (SMEqr / σ_max))
    # print("--- %s seconds QR ---" % (time.time() - start_time))

    # SVD
    start_time = time.time()
    Xsvd = SVD(A, b, N + 1)
    print("Обусловленность А^t * A  %e" % np.linalg.cond(np.matmul(A.transpose(), A)) + "|||| SMEsvd = %e" % (SME(A, Xsvd, b, len(A), N) / σ_max))
    print("--- %s seconds SVD ---" % (time.time() - start_time))

    # НУ
    # start_time = time.time()
    # At = A.transpose()
    # AtA = np.matmul(At, A)
    # Xnu = np.linalg.solve(AtA, np.matmul(At, b))
    # print("Обусловленность А^t * A  %e" % np.linalg.cond(AtA) + "|||| SMEnu = %e" % (SME(A, Xnu, b, len(A), N) / σ_max))
    # print("--- %s seconds SME ---" % (time.time() - start_time))

    # Отрисовка
    if N < 3 :
        # plt.plot(ε, np.matmul(A, Xnu), label='NU Estimate N={}'.format(N))
        # plt.plot(ε, b, label='True values')
        # leg = plt.legend(loc='upper center')
        # plt.ylabel('y')
        # plt.xlabel('x')
        # plt.figure(dpi=1200)
        # plt.show()

        # plt.plot(ε, np.matmul(A, Xqr), label='QR Estimate N={}'.format(N))
        # plt.plot(ε, b, label='True values')
        # leg = plt.legend(loc='upper center')
        # plt.ylabel('y')
        # plt.xlabel('x')
        # plt.figure(dpi=1200)
        # plt.show()

        plt.plot(ε, np.matmul(A, Xsvd), label='SVD Estimate N={}'.format(N))
        plt.plot(ε, b, label='True values')
        leg = plt.legend(loc='upper center')
        plt.ylabel('y')
        plt.xlabel('x')
        plt.figure(dpi=1200)
        plt.show()

    if (N == 5) or (N == 7) or (N == 10):
        #plt.plot(ε, (np.matmul(A, Xqr) - b)**2, label='QR - TrueValue Estimate N={}'.format(N))
        #plt.plot(ε, (np.matmul(A, Xnu) - b)**2, label='NU - TrueValue Estimate N={}'.format(N))
        plt.plot(ε, (np.matmul(A, Xsvd) - b)**2, label='SVD - TrueValue Estimate N={}'.format(N))
        leg = plt.legend(loc='upper center')
        plt.ylabel('y')
        plt.xlabel('x')
        plt.figure(dpi=1200)
        plt.show()